In [0]:
from pathlib import Path

# === ① Bronze 側のパスを定義 ===========================
bronze_pdf_root = "dbfs:/bronze/jsai2025/pdfs"
print(f"Bronze 側 PDF ディレクトリ: {bronze_pdf_root}")

# Bronze 側ディレクトリ作成（無ければ作成、あれば何もしない）
dbutils.fs.mkdirs(bronze_pdf_root)

# === ② repo ルートを特定（このノートブックの 1 つ上のディレクトリ） ===
cwd = Path.cwd()
repo_root = cwd.parent
print(f"現在の作業ディレクトリ (cwd): {cwd}")
print(f"推定 repo ルート: {repo_root}")

# === ③ repo ルート直下の dataset/pdfs を想定 =========================
pdf_dir = repo_root / "dataset" / "pdfs"
print(f"想定している PDF ディレクトリ: {pdf_dir}")

if not pdf_dir.exists():
    raise FileNotFoundError(
        "想定している PDF ディレクトリが存在しませんでした。\n"
        f"  想定パス: {pdf_dir}\n"
        "→ Repos 上のフォルダ構成を確認してください。\n"
        "  例) demo-resonac-databricks-script/dataset/pdfs に PDF があるか？"
    )

print("✅ PDF ディレクトリが見つかりました。")


In [0]:
# pdf_dir, bronze_pdf_root は Cell1 で定義されている前提

# === ④ Repos 側の PDF を列挙 ===========================
pdf_files = list(pdf_dir.glob("*.pdf"))

print(f"検出した PDF ファイル数: {len(pdf_files)}")
for p in pdf_files[:10]:
    print(f"  - {p.name}")

if not pdf_files:
    raise FileNotFoundError("dataset/pdfs 配下に PDF が1つもありません。")


# === ⑤ 既に Bronze にあるファイル名を収集 =================
bronze_existing = {f.name for f in dbutils.fs.ls(bronze_pdf_root)}
print(f"Bronze 側に既に存在するファイル数: {len(bronze_existing)}")

# === ⑥ ローカル(Repos) → DBFS(Bronze) にコピー =================
copied = 0
skipped = 0

for p in pdf_files:
    dst = f"{bronze_pdf_root}/{p.name}"

    if p.name in bronze_existing:
        print(f"SKIP (already exists): {p.name}")
        skipped += 1
        continue

    src = f"file:{p}"  # Repos(ローカル FS) のパスを file: スキームで指定
    print(f"COPY: {src} -> {dst}")
    dbutils.fs.cp(src, dst)
    copied += 1

print("==== コピー結果 ====")
print(f"  コピー: {copied} ファイル")
print(f"  スキップ: {skipped} ファイル")


In [0]:
display(dbutils.fs.ls(bronze_pdf_root))